# QUEUES

## NOTE: this needs to be run in docker because graphviz is a pain to install in OS X




## 004. THIS IS ABOUT...

This is part 2 of https://realpython.com/queue-in-python/

In [2]:
%load_ext nb_mypy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # type: ignore

Version 1.0.5


In [3]:
from collections import deque
from heapq import heappop, heappush
import networkx as nx
from typing import Any, Callable, Deque, Generic, NamedTuple, TypeVar
from enum import IntEnum

In [4]:

# assets from part 1

T = TypeVar('T')

class IterableMixin:
    def __len__(self) -> int:
        return len(self._elements) # type: ignore

    def __iter__(self):
        while len(self) > 0:
            yield self.dequeue()

class Queue(IterableMixin, Generic[T]):
    def __init__(self, *elements: T):
        self._elements = deque(elements)

    def enqueue(self, element:T):
        self._elements.append(element)

    def dequeue(self) -> T:
        return self._elements.popleft()

class Stack(Queue, Generic[T]):
    def dequeue(self) -> T:
        return self._elements.pop()

roadmap_file = "004_roadmap.dot"

graph = nx.nx_agraph.read_dot(roadmap_file)

class City(NamedTuple):
    name: str
    country: str
    year: int | None
    latitude: float
    longitude: float

    @classmethod
    def from_dict(cls, attrs:dict[str, Any]) -> "City":
        return cls(
            name=attrs["xlabel"],
            country=attrs["country"],
            year=int(attrs["year"]) or None,
            latitude=float(attrs["latitude"]),
            longitude=float(attrs["longitude"]),
        )

def as_cities_map(
        agraph: nx.Graph,
        node_factory: Callable[[dict[str, Any]], Any]
    ) -> dict[str, Any]:
    return {
        name: node_factory(attributes)
        for name, attributes in agraph.nodes(data=True)
    }

def as_cities_graph(cities_map:dict[str, City], agraph: nx.Graph) -> nx.Graph:
    return nx.Graph(
        (cities_map[name1], cities_map[name2], weights)
        for name1, name2, weights in agraph.edges(data=True)
    )

### 001.006 Breadth-First Search Using a FIFO Queue

In the breadth-first search algorithm, you look for a node that satisfies a particular condition by exploring the graph in concentric layers or levels. The breadth-first search algorithm ensures that you’ll eventually explore all connected nodes in a graph while searching for one that satisfies the desired condition. You start traversing the graph at an arbitrarily chosen source node unless the graph is a tree data structure, in which case you typically start at the root node of that tree. At each step, you visit all immediate neighbors of the current node before going deeper. 

1. Reuse `as_cities_map` and `as_cities_graph`
2. nx has a breadth-first search function, `nx.bfs_tree`. Use it to identify the closest neighbour of `CITY` which was granted city status in the XXth century
    1. Use `print_name` to show what the loop is doing
    1. Use the `is_twentieth_century` to find the city and print 
3. The actual order of the neighbours is unknown unless you specify. Let's do that
    1. function `order` does the orderinging
        1. it returns an iterable...
        1. ...and orders by latitude (could be anything really)
    1. add that ordering to the `nx.bfs_tree` function
4. Use the Queue from earlier to implement your own BFS
    1. Create `breadth_first_traverse` which does breadth-first traverse of QUEUE
        1. Initialise the FIFO queue with source
        1. To ensure no infinite loops, keep track of visited nodes and skip them
        1. `breadth_first_traverse` is an iterator that returns the latest dequeued node
            1. (HINT: walrus operator)
        1. once a node is dealt with, add its neightbours to the queue
    1. Create `breadth_first_search` which iterates through `breadth_first_traverse` and returns the first node that matches
    1. call `breadth_first_search` with the same parameters as the `nx.bfs_tree` version and check that results match
5. Add the `order_by` clause, same as `nx.bfs_tree` example, and call function again using `order` for it. It should match previous example
    1. Note that the order_by function is slightly different - it takes a single City, as we are using simple list sorting
    1. order has to be passed to search function too


In [5]:

# from earlier
1
cities_map = as_cities_map(graph, City.from_dict)
cities_graph = as_cities_graph(cities_map, graph)

CITY = "edinburgh"

def print_name(node):
    print("📍", node.name)

def print_found(node):
    print("Found:", node.name, node.year)

def is_twentieth_century(node):
    return node.year and 1901 <= node.year <= 2000

# 2
# for node in nx.bfs_tree(...):
#     2.1
#     if 2.2
#         print_found(node)
# else:
#     print("Not found")

# 3.1
# def order(neighbors):
#     def by_latitude(city):
#         return city.latitude
#     return ...(sorted(neighbors, key=..., reverse=True))

# 4.1
# def breadth_first_traverse(graph, source):
#     queue = 4.1.1
#     visited = 4.1.2
#     while queue:
#         4.1.3
#         for neighbor in graph.neighbors(node):
#             4.1.4

# 4.2
# def breadth_first_search(graph, source, predicate):
#     ...

# 4.3
# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

5
# def order_by_lat(neighbor: City) -> float:
#     ...

# def breadth_first_traverse(graph, source, order_by=None):
#     ...

# def breadth_first_search(graph, source, order_by=None):
#     ...

# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century, order_by_lat)

# solution

2
for node in nx.bfs_tree(G=cities_graph, source=cities_map[CITY]):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

3
# 3.1
def sort_neighbors(neighbors:list[City]):
    def by_latitude(city):
        return city.latitude
    return iter(sorted(neighbors, key=by_latitude, reverse=True))

3.2
for node in nx.bfs_tree(G=cities_graph, source=cities_map[CITY], sort_neighbors=sort_neighbors):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

4

def breadth_first_traverse(graph, source):
    queue = Queue(source)
    visited = {source}
    while queue:
        yield (node := queue.dequeue())
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)

def breadth_first_search(graph, source, predicate):
    for node in breadth_first_traverse(graph, source):
        if predicate(node):
            return node

breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

5
def order_by(neighbor: City) -> float:
    return -neighbor.latitude

def breadth_first_traverse_2(graph:nx.Graph, source:City, order_by=None):
    queue = Queue(source)
    visited = {source}
    while queue:
        yield (node := queue.dequeue())
        neighbors:list[City] = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)

def breadth_first_search_2(graph:nx.Graph, source:City, predicate, order_by=None):
    for node in breadth_first_traverse_2(graph, source, order_by):
        if predicate(node):
            return node

breadth_first_search_2(cities_graph, cities_map[CITY], is_twentieth_century, order_by_lat)


1

5

2

📍 Edinburgh
📍 Dundee
📍 Glasgow
📍 Perth
📍 Stirling
📍 Carlisle
📍 Newcastle upon Tyne
📍 Aberdeen
📍 Inverness
📍 Lancaster
Found: Lancaster 1937


3

...

📍 Edinburgh
📍 Dundee
📍 Perth
📍 Stirling
📍 Glasgow
📍 Newcastle upon Tyne
📍 Carlisle
📍 Aberdeen
📍 Inverness
📍 Sunderland
Found: Sunderland 1992


4

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)

5

City(name='Sunderland', country='England', year=1992, latitude=54.91, longitude=-1.385)

### 001.007 Shortest Path Using Breadth-First Traversal

In many cases, the fewer the nodes on the path from source to destination, the shorter the distance. Traversing the graph using the breadth-first approach will produce a path guaranteed to have the fewest nodes.

1. Use networkx to reveal all the shortest paths between two cities, which will have the same minimal length
    1. Import cities_map and cities_graph as earlier
    1. Create two cities nodes for aberdeen and perth
    1. nx has a builtin method for that, use it. Start enumerator from 1 and not the default 0
1. Write your own version, `shortest_path` (which only finds the first shortest path)
    1. Base it on `breadth_first_traverse_2`, but keep a dictionary of previous nodes
        1. Keep a trail of nodes in `previous_nodes`
        1. When you find the destination, retrace the trail with a new function `retrace` (next step)
    1. `retrace` takes a dictionary of City -> City and returns a path through them
        1. Store the results in a standard Deque, no need for more
        1. load the destination in a var `current`, and loop through `previous_nodes` to find the trail.
        1. Unshift into the deque as you find it
        1. Ends when the `current` is either None or the source
    1. Call the function, should be the same as one of the paths
        1. Note that you get a different result if you order nodes differently
1. `shortest_path` can also be used to find out whether two nodes are connected or not


In [6]:
# from earlier
# 1.1
cities_map = as_cities_map(graph, City.from_dict)
cities_graph = as_cities_graph(cities_map, graph)

# 1.2
city1 = cities_map["aberdeen"]
city2 = cities_map["perth"]

# 1.3
# for i, path in enumerate(nx.all_shortest_paths..., ...):
#     print(f"{i}.", " → ".join(city.name for city in path))

# 2.1
# def shortest_path(
#         graph:nx.Graph, source:City, destination:City, order_by=None
#     ) -> list[City]:
#     queue = Queue(source)
#     visited = {source}
#     previous_nodes = ...
#     while queue:
#         node = queue.dequeue()
#         neighbors:list[City] = list(graph.neighbors(node))
#         if order_by:
#             neighbors.sort(key=order_by)
#         for neighbor in neighbors:
#             if neighbor not in visited:
#                 visited.add(neighbor)
#                 queue.enqueue(neighbor)
#                 # ... 2.1.1
#                 # ...
#                     return retrace(previous_nodes, source, destination)
#     return []

# 2.2
def retrace(
    previous_nodes:dict[City, City], source:City, destination:City
    ) -> list[City]:
    return []

# 2.3
def by_latitude(city:City):
    return -city.latitude
# " → ".join(city.name for city in shortest_path(cities_graph, city1, city2))
# " → ".join(city.name for city in shortest_path(cities_graph, city1, city2, by_latitude))

# 3
def connected(graph:nx.Graph, source:City, destination:City) -> bool:
    return True

# assert connected(cities_graph, cities_map["belfast"], cities_map["glasgow"]) == False
# assert connected(cities_graph, cities_map["belfast"], cities_map["derry"]) == True

# solution

1.3
for i, path in enumerate(nx.all_shortest_paths(cities_graph, city1, city2), 1):
    print(f"{i}.", " → ".join(city.name for city in path))

2.1
def shortest_path(
        graph:nx.Graph, source:City, destination:City, order_by=None
    ) -> list[City]:
    queue = Queue(source)
    visited = {source}
    previous_nodes = {}
    while queue:
        node = queue.dequeue()
        neighbors:list[City] = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)
                previous_nodes[neighbor] = node
                if neighbor == destination:
                    return retrace(previous_nodes, source, destination)
    return []

2.2
def retrace( # type: ignore[no-redef]
    previous_nodes:dict[City, City], source:City, destination:City
    ) -> list[City]:
    path:Deque[City] = deque()

    current: City | None = destination
    while current and current != source:
        path.appendleft(current)
        current = previous_nodes.get(current)
        if current is None:
            return []

    path.appendleft(source)
    return list(path)

2.3
" → ".join(city.name for city in shortest_path(cities_graph, city1, city2))
" → ".join(city.name for city in shortest_path(cities_graph, city1, city2, by_latitude))


# 3
def connected(graph:nx.Graph, source:City, destination:City) -> bool: # type: ignore[no-redef]
    return shortest_path(graph, source, destination) != []

assert connected(cities_graph, cities_map["belfast"], cities_map["glasgow"]) == False
assert connected(cities_graph, cities_map["belfast"], cities_map["derry"]) == True

1.3

1. Aberdeen → Dundee → Perth
2. Aberdeen → Inverness → Perth


2.1

2.2

2.3

'Aberdeen → Dundee → Perth'

'Aberdeen → Inverness → Perth'

### 001.008 Depth-First Search Using a LIFO Queue

The depth-first traversal follows a path from the source node by plunging into the graph as deeply as possible before backtracking to the last edge crossing and trying another branch

1. You'll be reusing `cities_map`, `is_twentieth_century`, etc from earlier
1. Use the built-in method from nx for it, same loop as before
1. `search` is a generic function that takes any traversal function and runs it, it will be used later
1. Write your own `depth_first_traverse`, loosely based on `breadth_first_traverse` but
    1. Use a stack instead of a Queue
    1. What is `visited` inited with? 
    1. There is a condition associated with yielding the node...
    1. The order of when things are added to `visited` is different
    1. Use `search` to test it works
1. You can use python's own call stack instead of creating your own, by writing a recursive version of `depth_first_traverse`.
    1. Use `search` to test it works



In [7]:

# from earlier

CITY = "edinburgh"

2
# for node in nx...
#     print_name(node)
#     if is_twentieth_century(node):
#         print_found(node)
#         break
# else:
#     print("Not found")


3
def search(traverse, graph:nx.Graph, source:City, predicate, order_by=None):
    for node in traverse(graph, source, order_by):
        if predicate(node):
            return node

3
# def depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
#     stack:Stack[City] = Stack(source)
#     visited = ... 3.2
#     while stack:
#         if (node := stack.dequeue()) ...3.3:
#             yield node
#             3.4
#             neighbors = list(graph.neighbors(node))
#             if order_by:
#                 neighbors.sort(key=order_by)
#             for neighbor in reversed(neighbors):
#                 3.4
# 3.5
# search(depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

4
# def recursive_depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
#     visited = set()

#     def visit(node):
#         yield node
#         ...

#     return visit(source)

# 4.1
# search(recursive_depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

# solution

2
for node in nx.dfs_tree(G=cities_graph, source=cities_map[CITY]):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

3
def depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
    stack:Stack[City] = Stack(source)
    visited = set()
    while stack:
        if (node := stack.dequeue()) not in visited:
            yield node
            visited.add(node)
            neighbors = list(graph.neighbors(node))
            if order_by:
                neighbors.sort(key=order_by)
            for neighbor in reversed(neighbors):
                stack.enqueue(neighbor)

3.5
search(depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

4
def recursive_depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
    visited = set()

    def visit(node):
        yield node
        visited.add(node)
        neighbors = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                yield from visit(neighbor)

    return visit(source)

4.1
search(recursive_depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

# def breadth_first_search(graph, source, predicate):
#     for node in breadth_first_traverse(graph, source):
#         if predicate(node):
#             return node

# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

# 5
# def order_by(neighbor: City) -> float:
#     return -neighbor.latitude

# def breadth_first_traverse_2(graph:nx.Graph, source:City, order_by=None):
#     queue = Queue(source)
#     visited = {source}
#     while queue:
#         yield (node := queue.dequeue())
#         neighbors:list[City] = list(graph.neighbors(node))
#         if order_by:
#             neighbors.sort(key=order_by)
#         for neighbor in neighbors:
#             if neighbor not in visited:
#                 visited.add(neighbor)
#                 queue.enqueue(neighbor)

# def breadth_first_search_2(graph:nx.Graph, source:City, predicate, order_by=None):
#     for node in breadth_first_traverse_2(graph, source, order_by):
#         if predicate(node):
#             return node

# breadth_first_search_2(cities_graph, cities_map[CITY], is_twentieth_century, order_by)


2

3

3

4

2

...

3.5

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)

4

4.1

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)